In [1]:
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO,DDPG,DQN
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import numpy as np
import time
from stable_baselines3.common.callbacks import EvalCallback
import env
env = env.ElectricVehicleEnv()
#vec_env = make_vec_env(env, n_envs=4)
check_env(env)
env = DummyVecEnv([lambda: env])

In [2]:
# 设置评估频率
eval_freq = 1000  # 每训练500步评估一次

# 配置EvalCallback
eval_callback = EvalCallback(
    env,
    callback_on_new_best=None,
    n_eval_episodes=20,
    best_model_save_path="./best_model/DQN",  # 更改保存路径为DQN
    log_path="./logs/DQN",  # 更改日志路径为DQN
    eval_freq=eval_freq,
)

# DQN的超参数设置
model = DQN(
    'MlpPolicy', 
    env, 
    verbose=0, 
    tensorboard_log="./logs/DQN",
    learning_rate=0.0005,  # 默认学习率，通常比A2C更小
    buffer_size=100000,  # 经验回放缓冲区大小
    learning_starts=1000,  # 开始学习前的步数
    batch_size=32,  # 批量大小
    tau=1.0,  # 目标网络软更新的系数
    gamma=0.99,  # 折扣因子
    train_freq=4,  # 训练的频率（每4步训练一次）
    target_update_interval=1000,  # 目标网络更新的频率
    exploration_fraction=0.1,  # 探索率衰减部分
    exploration_final_eps=0.02,  # 最小探索率
    max_grad_norm=10,  # 梯度裁剪的最大范数
)


# 训练模型，并将TensorBoard日志存储到log目录中
model.learn(total_timesteps=1000000, tb_log_name="DQN", callback=eval_callback)

# 保存模型
model.save("model/dqn_electric_vehicle")

# 加载模型（如果需要）
# model = DQN.load("dqn_electric_vehicle")

# 评估模型
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# 测试训练好的模型
obs = env.reset()
for i in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

Eval num_timesteps=1000, episode_reward=-8328.59 +/- 112.33
Episode length: 2.95 +/- 0.80
New best mean reward!
Eval num_timesteps=2000, episode_reward=-8167.17 +/- 68.16
Episode length: 2.00 +/- 0.00
New best mean reward!
Eval num_timesteps=3000, episode_reward=-7356.18 +/- 2432.24
Episode length: 1.90 +/- 0.30
New best mean reward!
Eval num_timesteps=4000, episode_reward=-6578.96 +/- 3258.64
Episode length: 2.25 +/- 0.77
New best mean reward!
Eval num_timesteps=5000, episode_reward=-8177.62 +/- 75.69
Episode length: 2.00 +/- 0.00
Eval num_timesteps=6000, episode_reward=-7901.77 +/- 1802.38
Episode length: 2.75 +/- 1.04
Eval num_timesteps=7000, episode_reward=-8225.68 +/- 67.71
Episode length: 2.40 +/- 0.66
Eval num_timesteps=8000, episode_reward=-8274.64 +/- 117.98
Episode length: 2.70 +/- 0.78
Eval num_timesteps=9000, episode_reward=-5777.72 +/- 3744.31
Episode length: 2.05 +/- 0.97
New best mean reward!
Eval num_timesteps=10000, episode_reward=-7057.56 +/- 2941.81
Episode length: 2